<a href="https://githubtocolab.com/giswqs/geemap/blob/master/examples/notebooks/114_dynamic_world.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

**Creating near real-time global 10-m land cover maps with geemap and Dynamic World**

y. agbohessou

- App: https://www.dynamicworld.app
- App2: https://earthoutreach.users.earthengine.app/view/dynamicworld
- Paper: https://doi.org/10.1038/s41597-022-01307-4
- Model: https://github.com/google/dynamicworld
- Training data: https://doi.pangaea.de/10.1594/PANGAEA.933475
- Data: https://developers.google.com/earth-engine/datasets/catalog/GOOGLE_DYNAMICWORLD_V1
- JavaScript tutorial: https://developers.google.com/earth-engine/tutorials/community/introduction-to-dynamic-world-pt-1

In [5]:
#pip install geemap


In [1]:
import ee
import geemap

In [2]:
Map = geemap.Map()
Map.add_basemap('HYBRID')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [3]:
# Set the region of interest by simply drawing a polygon on the map
region = Map.user_roi
if region is None:
    #region = ee.Geometry.BBox(-17.6, 12.3, -11.3, 16.7)# Senegal
    region = ee.Geometry.BBox(-18, 13, 20, 18)# Sahel

Map.centerObject(region)

In [4]:
# Set the date range
start_date = '2021-08-01'
end_date = '2021-11-30'

In [5]:
# Create a Sentinel-2 image composite
image = geemap.dynamic_world_s2(region, start_date, end_date)
vis_params = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}
Map.addLayer(image, vis_params, 'Sentinel-2 image')

In [6]:
# Create Dynamic World land cover composite
landcover = geemap.dynamic_world(region, start_date, end_date, return_type='hillshade')
# hillshade to see the highs (high accuracy) and the lows (low accuracy) when we zoom the img. 
#every pixel is a probability of classification
#landcover = geemap.dynamic_world(region, start_date, end_date, return_type='visualize')
Map.addLayer(landcover, {}, 'Land Cover')

In [7]:
# visualize the classes with the original imagery
landcover = geemap.dynamic_world(region, start_date, end_date, return_type='class')
vis = {'min':0, 'max':8, 'palette':  [
    '#419BDF', '#397D49', '#88B053', '#7A87C6', '#E49635', '#DFC35A',
    '#C4281B', '#A59B8F', '#B39FE1'
  ]}
Map.addLayer(landcover, vis, 'Land Cover')

# in the setting, click inspector and then click on a pixel to see its class

In [34]:
# get the probability that a pixel is well classified 
landcover = geemap.dynamic_world(region, start_date, end_date, return_type='probability')
vis = {'min':0, 'max':8, 'palette':  [
    '#419BDF', '#397D49', '#88B053', '#7A87C6', '#E49635', '#DFC35A',
    '#C4281B', '#A59B8F', '#B39FE1'
  ]}
Map.addLayer(landcover, {}, 'Land Cover')

# in the setting, click inspector and then click on a pixel to see its class

In [8]:
# to download the image
#geemap.ee_export_image(landcover,"landcover.tif",scale=100) #for small area
geemap.ee_export_image_to_drive(landcover,"sahel_landcover_wet_season_2021_08_to_11.tif",scale=100) #for large area 

Exporting sahel_landcover_wet_season_2021_08_to_11.tif ...


In [66]:
# Add legend to the map
Map.add_legend(title="Dynamic World Land Cover", builtin_legend='Dynamic_World')
Map

Map(bottom=15326.800048828125, center=[14.745135361581408, -13.176648223577743], controls=(WidgetControl(optio…

![](https://i.imgur.com/GEzsSii.png)